In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from time import time
import io
import re

import pickle
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import plot_model

# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

from datetime import datetime
from collections import defaultdict

from sklearn.model_selection import train_test_split



# Data

In [2]:
train_section = pd.read_table("Article-Bias-Prediction-main/data/randomtraindata.csv", sep=',', index_col=0)
test_section = pd.read_table("Article-Bias-Prediction-main/data/randomtestdata.csv", sep=',', index_col=0)
val_section = pd.read_table("Article-Bias-Prediction-main/data/randomtestdata.csv", sep=',', index_col=0)


In [3]:
test_section.head()

,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,palestine,Breitbart News,2,http://www.breitbart.com/big-journalism/2015/0...,"Obama, Media Lied About Netanyahu and the Pale...",2015-03-20,Ben Shapiro,"For the last several days , the entire Western...","For the last several days, the entire Western ...",www.breitbart.com,right,uI2K5BSCO0LaWuKb
1,federal_budget,ABC News,0,http://abcnews.go.com/blogs/politics/2013/03/w...,Will Fractured House Republicans Unite on Budget?,NaN,John Parkinson,"Tomorrow morning , House Budget Chairman Paul ...","Jacquelyn Martin/AP Photo\n\nTomorrow morning,...",www.abcnews.go.com,left,zeSfcxLilzR0R1Dt
2,fbi,Reuters,1,https://www.reuters.com/article/us-usa-trump-r...,Trump blocks release of Russia memo drafted by...,2018-02-10,Ayesha Rascoe,WASHINGTON ( ███ ) - President Donald Trump on...,WASHINGTON (Reuters) - President Donald Trump ...,www.reuters.com,center,ru8kAxfdPzZc11le
3,politics,CBN,2,http://www1.cbn.com/cbnnews/politics/2019/febr...,Trump's North Korea Peace Talks Share Spotligh...,2019-02-25,NaN,"President Donald Trump arrives in Hanoi , Viet...","President Donald Trump arrives in Hanoi, Vietn...",www1.cbn.com,right,TTt5FFmsW6ng3fsC
4,media_bias,Breitbart News,2,http://www.breitbart.com/big-government/2017/0...,Six Times President Trump Upset Jared Kushner ...,2017-08-20,Adam Shaw,President Trump ’ s fiery response to the even...,President Trump’s fiery response to the events...,www.breitbart.com,right,gYKaZGjLCYdEghTZ


In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_labels = np.asarray(le.fit_transform(train_section['bias_text']))
val_labels = np.asarray(le.transform(val_section['bias_text']))
test_labels = np.asarray(le.transform(test_section['bias_text']))

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

## TF-IDF Label Prediction

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
# tf-idf processing from https://github.com/williamscott701/Information-Retrieval/blob/master/2.%20TF-IDF%20Ranking%20-%20Cosine%20Similarity%2C%20Matching%20Score/TF-IDF.ipynb

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text


def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text


def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text


def preprocess(data):
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [8]:
train_dataset = list(train_section["content"])
val_dataset = list(val_section["content"])
test_dataset = list(test_section["content"])


In [9]:
def process(dataset):
    processed_text = []
    for i in dataset:
#         processed_text.append(word_tokenize(str(preprocess(i))))
        processed_text.append(str(preprocess(i)))
    return processed_text


In [10]:
train_dataset = process(train_dataset)
val_dataset = process(val_dataset)
test_dataset = process(test_dataset)

In [11]:
# tf-idf from https://stackabuse.com/text-classification-with-python-and-scikit-learn/
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
trainX = vectorizer.fit_transform(train_dataset).toarray()
valX = vectorizer.transform(val_dataset).toarray()
testX = vectorizer.transform(test_dataset).toarray()

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
trainX = tfidfconverter.fit_transform(trainX).toarray()
valX = tfidfconverter.transform(valX).toarray()
testX = tfidfconverter.transform(testX).toarray()

In [ ]:

# train_labels = np.asarray(le.transform(train_section['label']))
# val_labels = np.asarray(le.transform(val_section['label']))
# test_labels = np.asarray(le.transform(test_section['label']))

In [25]:
trainX.shape

(27978, 1500)

In [26]:
valX.shape

(1300, 1500)

In [27]:
trainvalX = np.concatenate((trainX, valX), axis=0)
trainvalX.shape

(29278, 1500)

In [30]:
trainval_labels = np.concatenate((train_labels, val_labels), axis=0)
trainval_labels.shape

(29278,)

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# param_grid = {
#                  'n_estimators': [100, 500, 1000],
#                  'max_depth': [2, 5, 9, 13]
#              }

param_grid = {
                 'n_estimators': [1000, 3000],
                 'max_depth': [2, 5, 13]
             }

rfc = RandomForestClassifier()

clf = GridSearchCV(rfc, param_grid, cv = 10)
clf.fit(trainX, train_labels)
sorted(clf.cv_results_.keys())
# classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# classifier.fit(trainX, train_labels) 

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_max_depth',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'split5_test_score',
 'split6_test_score',
 'split7_test_score',
 'split8_test_score',
 'split9_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [35]:
clf.best_params_

{'max_depth': 13, 'n_estimators': 3000}

In [14]:
import pickle

In [45]:
# https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn
pickle.dump(train_dataset, open("processed_train.pickle", "wb"))
pickle.dump(val_dataset, open("processed_val.pickle", "wb"))
pickle.dump(test_dataset, open("processed_sourcetest.pickle", "wb"))

In [46]:
pickle.dump(trainX, open("transformed_train.pickle", "wb"))
pickle.dump(valX, open("transformed_val.pickle", "wb"))
pickle.dump(testX, open("transformed_sourcetest.pickle", "wb"))

In [36]:
pickle.dump(clf, open("clfsourcelevel.pickle", "wb"))

In [37]:
pickle.dump(vectorizer, open("vectorizersourcelevel.pickle", "wb"))
pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))
pickle.dump(tfidfconverter, open("tfidfconverter.pickle", "wb"))

array([2, 2, 0, ..., 1, 2, 0])

In [39]:
test_labels

array([2, 1, 0, ..., 2, 2, 0])

#### TF-IDF Label Prediction

In [40]:
y_pred = clf.predict(testX)
y_pred

array([2, 2, 0, ..., 1, 2, 0])

In [41]:
test_labels

array([2, 1, 0, ..., 2, 2, 0])

In [44]:
pickle.dump(y_pred, open("pred_y_labels_TFIDF_source.pickle", "wb"))
pickle.dump(test_labels, open("real_y_labels_TFIDF_source.pickle", "wb"))


In [42]:
test_classes = y_pred
modelname = "TF-IDF"

print(modelname +" Metrics:")
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_labels, test_classes)
print('Accuracy: %f' % accuracy)
# modelaccuracylist.append(accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_labels, test_classes, "micro")
print('Precision (micro): %f' % precision)
# modelprecisionlist.append(precision)
# recall: tp / (tp + fn)
recall = recall_score(test_labels, test_classes)
print('Recall: %f' % recall)
# modelrecalllist.append(recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_labels, test_classes)
# modelf1list.append(f1)
print('F1 score: %f' % f1)
# mlist.append([accuracy, precision, recall, f1])
#     print(classification_report(test_labels,test_classes))
print("")

TF-IDF Metrics:
Accuracy: 0.679231


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].